<a href="https://colab.research.google.com/github/JoacoPagani/Data-Science-II-Machine-Learning-para-la-Ciencia-de-Datos/blob/main/EntregaFinalP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡¡ Debido a una constante caida del Entorno colab dado que la RAM del sistema se saturaba se tuvo que tomar la medida de separar la entrega en 2 archivos .ipynb, donde el ultimo codigo del anterior es la descarga del Df el cual se sigue trabajando aca !!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('df.csv')

#Previo al Entrenamiento

####Separacion de la variable

Debido a que queremos predecir la variable "Price" dividiremos el DataSet en las variables dependientes y independientes

In [5]:
Y_df = df[['price']].copy()
X_df = df.drop('price', axis=1).copy()

Por razones que exceden fue necesario tomar unicamente una parte de los datos, debido a que no era posible entrenar a los modelos

In [6]:
X_df_sampled = X_df.sample(frac=0.3, random_state=42)
Y_df_sampled = Y_df.loc[X_df_sampled.index]

####Eliminacion de variables correlativas

Durante el EDA se realizo la matriz de coeficientes de Correlación de Pearson se encontro una correlacion moderada-fuerte entre las variables "surface_total" y "surface_covered". Por lo que se decide eliminar una de las dos para mejorar la predicción.
Ademas se eliminan las variables "price_period" y "operation_type" ya que todas las filas contaban con el mismo valor. Y la variable "title" que no tiene aporte en el entrenamiento del modelo

In [7]:
X_df_sampled.drop(['surface_covered', 'price_period', 'title', 'operation_type'], axis=1, inplace=True)

####Estandarizacion de variables

Estandarizamos las variables numericas ya que ayuda a mejorar la estabilidad numérica, la convergencia del modelo y la interpretación de los resultados

In [8]:
from sklearn.preprocessing import StandardScaler
# Identificar las columnas numéricas
numerical_cols = X_df_sampled.select_dtypes(include=['number']).columns

# Excluir las columnas de longitud y latitud de la lista de columnas numéricas
numerical_cols = [col for col in numerical_cols if col not in ['lat', 'lon']]

# Reemplazar valores infinitos por NaN
X_df_sampled[numerical_cols] = X_df_sampled[numerical_cols].replace([np.inf, -np.inf], np.nan)

# Rellenar NaN con la media de cada columna
for col in numerical_cols:
    X_df_sampled[col] = X_df_sampled[col].fillna(X_df_sampled[col].mean())

# Instanciar el objeto scaler
scaler = StandardScaler()

# Aplicar la transformación solo a las columnas seleccionadas
X_df_sampled[numerical_cols] = scaler.fit_transform(X_df_sampled[numerical_cols])


Para poder aplicar los modelos de Machine Learning, transformamos las variables categóricas aplicando la codificación One Hot Encoding.

In [9]:
X_df_sampled = pd.get_dummies(X_df_sampled, columns=X_df_sampled.select_dtypes(include=['object']).columns, drop_first=True)

X_df_sampled = X_df_sampled.astype(float)

X_df_sampled

,lat,lon,rooms,bedrooms,bathrooms,surface_total,price_m2,created_on_2018-12-30,created_on_2018-12-31,created_on_2019-01-01,...,l3_Ángel Gallardo,property_type_Casa de campo,property_type_Cochera,property_type_Departamento,property_type_Depósito,property_type_Local comercial,property_type_Lote,property_type_Oficina,property_type_Otro,property_type_PH
32351,-38.032001,-57.563171,1.380869,-0.108426,-0.783203,-0.086750,-8.179040e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36813,-34.593887,-58.492172,-0.005689,-0.108426,0.028045,-0.053997,-1.208944e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
46729,-34.656774,-58.559328,-0.005689,-0.108426,-0.783203,-0.112201,4.282077e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
51260,-36.698823,-56.677481,-0.769369,-0.108426,-0.783203,-0.092283,1.694616e-16,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50826,-26.909278,-65.684205,-0.005689,-0.108426,0.028045,-0.092283,1.694616e-16,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33631,-34.568378,-58.489971,-0.769369,-0.108426,-0.783203,-0.106668,7.031466e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39950,-38.012192,-57.537347,-0.052623,-0.108426,0.334572,-0.102242,5.236358e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2052,-34.564340,-58.458355,-0.005689,-0.108426,0.028045,-0.114857,2.001265e-01,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49342,-34.601071,-58.375274,-0.005689,-0.108426,-0.783203,-0.109324,2.003028e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#Construccion de Modelos

LLevaremos a cabo los entrenamientos y puesta en prueba del rendimiento de diferentes tipos de modelos.

###Modelo 1 - Random Forest   



Dividimos los datos como primer paso, definiendo el conjunto de entrenamiento


In [14]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_df_sampled, Y_df_sampled, test_size=0.2, random_state=123)

#Veamos como quedaron divididos los conjuntos
X_train.shape, X_test.shape
"""
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_predRF = model.predict(X_test)"""

'\nmodel = RandomForestRegressor(n_estimators=100, random_state=42)\nmodel.fit(X_train, y_train)\ny_predRF = model.predict(X_test)'

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE
mae = mean_absolute_error(y_test, y_predRF)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate MSE
mse = mean_squared_error(y_test, y_predRF)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate R-squared
r2 = r2_score(y_test, y_predRF)
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 32107.795678166505
Mean Squared Error (MSE): 49483675699.10221
Root Mean Squared Error (RMSE): 222449.26544968004
R-squared (R2): 0.7106836050535907


### Modelo 2 - XGBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = RandomizedSearchCV(model, param_grid, cv=3, n_iter=20, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [ ]:
# prompt: analiza el accuracy de este model

y_predXGB = best_model.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_predXGB)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate MSE
mse = mean_squared_error(y_test, y_predXGB)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate R-squared
r2 = r2_score(y_test, y_predXGB)
print(f"R-squared (R2): {r2}")

Mean Absolute Error (MAE): 54432.73267340284
Mean Squared Error (MSE): 54740947845.005936
Root Mean Squared Error (RMSE): 233967.8350650062
R-squared (R2): 0.6799458861351013


###Modelo 3 - GradientBoostingRegressor

In [16]:
# Regression models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import RandomizedSearchCV

In [11]:
class BaseModel:
    """Clase base con funcionalidad común para modelos de clasificación y regresión"""
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.models = {}
        self.best_model = None
        self.model_scores = {}

    def save_model(self, filename):
        """Guarda un modelo en disco"""
        if self.best_model:
            with open(filename, 'wb') as f:
                pickle.dump(self.best_model, f)
            print(f"Modelo guardado como {filename}")

    @staticmethod
    def load_model(filename):
        """Carga un modelo desde disco"""
        with open(filename, 'rb') as f:
            return pickle.load(f)
    def save_best_model(self, filepath, save_scaler=True, model_info=True):
        """
        Guarda el mejor modelo junto con información relevante y el scaler si existe.

        Parameters:
        -----------
        filepath : str
            Ruta base donde se guardará el modelo (sin extensión)
        save_scaler : bool, default=True
            Si es True, guarda el scaler junto con el modelo (si existe)
        model_info : bool, default=True
            Si es True, guarda un archivo JSON con información sobre el modelo

        Returns:
        --------
        dict
            Diccionario con las rutas de los archivos guardados
        """
        import os
        import joblib
        import json
        from datetime import datetime

        saved_files = {}

        try:
            # Crear el directorio si no existe
            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            # Guardar el modelo
            model_path = f"{filepath}_model.joblib"
            joblib.dump(self.best_model, model_path)
            saved_files['model'] = model_path

            # Guardar el scaler si existe y se solicita
            if save_scaler and hasattr(self, 'scaler'):
                scaler_path = f"{filepath}_scaler.joblib"
                joblib.dump(self.scaler, scaler_path)
                saved_files['scaler'] = scaler_path

            # Guardar información del modelo
            if model_info:
                info = {
                    'model_type': type(self.best_model).__name__,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'feature_names': list(self.X_train.columns) if hasattr(self.X_train, 'columns') else None,
                    'parameters': self.best_model.get_params(),
                }

                # Agregar métricas específicas según el tipo de modelo
                if isinstance(self, RegressionModel):
                    best_scores = min(self.model_scores.items(), key=lambda x: x[1]['mse'])
                    info['metrics'] = {
                        'mse': best_scores[1]['mse'],
                        'mae': best_scores[1]['mae'],
                        'r2': best_scores[1]['r2']
                    }
                else:  # ClassificationModel
                    best_scores = max(self.model_scores.items(), key=lambda x: x[1]['accuracy'])
                    info['metrics'] = {
                        'accuracy': best_scores[1]['accuracy'],
                        'classification_report': best_scores[1]['classification_report']
                    }

                info_path = f"{filepath}_info.json"
                with open(info_path, 'w', encoding='utf-8') as f:
                    json.dump(info, f, indent=4)
                saved_files['info'] = info_path

            print(f"Modelo guardado exitosamente en: {model_path}")
            return saved_files

        except Exception as e:
            print(f"Error al guardar el modelo: {str(e)}")
            return None

In [12]:
class RegressionModel(BaseModel):
    def __init__(self, X_train, y_train, X_test, y_test):
        super().__init__(X_train, y_train, X_test, y_test)

    def train_models(self, custom_models=None):
        """Entrena múltiples modelos de regresión con búsqueda de hiperparámetros"""
        default_regressors = {
            'RandomForest': (RandomForestRegressor(), {
                'n_estimators': [50, 100, 200],
                'max_depth': [10, 20, 30],
                'min_samples_split': [2, 5, 10]
            }),
            'LinearRegression': (LinearRegression(), {}),
            'Ridge': (Ridge(), {
                'alpha': [0.1, 1.0, 10.0],
                'solver': ['auto', 'svd', 'cholesky']
            }),
            'Lasso': (Lasso(), {
                'alpha': [0.1, 1.0, 10.0],
                'selection': ['cyclic', 'random']
            }),
            'ElasticNet': (ElasticNet(), {
                'alpha': [0.1, 1.0, 10.0],
                'l1_ratio': [0.2, 0.5, 0.8]
            }),
            'SVR': (SVR(), {
                'C': [0.1, 1, 10],
                'kernel': ['rbf', 'linear'],
                'gamma': ['scale', 'auto']
            }),
            'GradientBoosting': (GradientBoostingRegressor(), {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.3],
                'max_depth': [3, 5, 7]
            }),
            'KNN': (KNeighborsRegressor(), {
                'n_neighbors': [3, 5, 7, 9],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']
            })
        }

        # Permite agregar modelos personalizados
        if custom_models:
            default_regressors.update(custom_models)

        for name, (reg, param_grid) in default_regressors.items():
            try:
                print(f"\nEntrenando {name}...")
                search = RandomizedSearchCV(
                    reg,
                    param_grid,
                    n_iter=10,
                    cv=5,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1
                )
                search.fit(self.X_train, self.y_train)
                self.models[name] = search.best_estimator_
                print(f"{name} - Mejores parámetros: {search.best_params_}")
                print(f"{name} - Mejor score CV: {-search.best_score_:.4f}")
            except Exception as e:
                print(f"Error entrenando {name}: {str(e)}")

    def evaluate_models(self):
        """Evalúa todos los modelos entrenados y guarda sus métricas"""
        best_score = float('inf')
        evaluation_results = []

        for name, model in self.models.items():
            try:
                y_pred = model.predict(self.X_test)
                mse = mean_squared_error(self.y_test, y_pred)
                mae = mean_absolute_error(self.y_test, y_pred)
                r2 = r2_score(self.y_test, y_pred)

                self.model_scores[name] = {
                    'mse': mse,
                    'mae': mae,
                    'r2': r2
                }

                evaluation_results.append({
                    'Model': name,
                    'MSE': mse,
                    'MAE': mae,
                    'R2': r2
                })

                if mse < best_score:
                    self.best_model = model
                    best_score = mse

                print(f"\nResultados para {name}:")
                print(f"MSE: {mse:.4f}")
                print(f"MAE: {mae:.4f}")
                print(f"R2: {r2:.4f}")

            except Exception as e:
                print(f"Error evaluando {name}: {str(e)}")

        # Crear DataFrame con resultados
        results_df = pd.DataFrame(evaluation_results)
        print("\nResumen de resultados:")
        print(results_df.sort_values('MSE'))
        print(f"\nMejor modelo: {type(self.best_model).__name__} con MSE: {best_score:.4f}")
        return results_df

In [ ]:
# Crear instancia
model = RegressionModel(X_train, y_train, X_test, y_test)

# Entrenar modelos con búsqueda de hiperparámetros
model.train_models()

# Evaluar modelos en el conjunto de prueba
results_df = model.evaluate_models()


Entrenando RandomForest...


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest - Mejores parámetros: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 30}
RandomForest - Mejor score CV: 69632047968.7784

Entrenando LinearRegression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


LinearRegression - Mejores parámetros: {}
LinearRegression - Mejor score CV: 8124739942104343744858556465152.0000

Entrenando Ridge...


...

Ridge - Mejores parámetros: {'solver': 'auto', 'alpha': 10.0}
Ridge - Mejor score CV: 120389388565.0788

Entrenando Lasso...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Lasso - Mejores parámetros: {'selection': 'random', 'alpha': 10.0}
Lasso - Mejor score CV: 126918256092.7684

Entrenando ElasticNet...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


ElasticNet - Mejores parámetros: {'l1_ratio': 0.8, 'alpha': 0.1}
ElasticNet - Mejor score CV: 120919291364.7032

Entrenando SVR...


In [ ]:


# Definir los hiperparámetros para el modelo
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5, 10]
}

# Instancia el modelo
model_gb = GradientBoostingRegressor(random_state=42)

# Realizar búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(estimator=model_gb, param_distributions=param_grid,
                                   n_iter=20, cv=3, scoring='neg_mean_absolute_error',
                                   random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_model_gb = random_search.best_estimator_


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score

# Evaluación con validación cruzada (usando MAE como métrica)
mae_cv_scores = cross_val_score(best_model_gb, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Cross-validated MAE: {-mae_cv_scores.mean():.2f}")


In [ ]:
# Predicción en el conjunto de prueba
y_predGB = best_model_gb.predict(X_test)

# Calcular métricas
mae = mean_absolute_error(y_test, y_predGB)
mse = mean_squared_error(y_test, y_predGB)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_predGB)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")
